In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json

# import kagglehub
# !kaggle datasets download -d adejumobidaniel/seq2seq-dataset
# !unzip /content/seq2seq-dataset.zip -d main_data

# import json

# with open('fr_str2int.json', 'w') as f:
#     json.dump(fr_str2int, f)

# with open('fr_int2str.json', 'w') as f:
#     json.dump(fr_int2str, f)

# with open('en_str2int.json', 'w') as f:
#     json.dump(en_str2int, f)

# with open('en_int2str.json', 'w') as f:
#     json.dump(en_int2str, f)

# !cp -r /content/main_data /content/drive/MyDrive
# !cp /content/en_int2str.json /content/drive/MyDrive/main_data
# !cp /content/en_str2int.json /content/drive/MyDrive/main_data
# !cp /content/fr_int2str.json /content/drive/MyDrive/main_data
# !cp /content/fr_str2int.json /content/drive/MyDrive/main_data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12

In [1]:
from datasets import load_from_disk
import json

dataset_train = load_from_disk ('/content/drive/MyDrive/main_data/dataset_train')
dataset_test = load_from_disk ('/content/drive/MyDrive/main_data/dataset_test')
dataset_validation = load_from_disk ('/content/drive/MyDrive/main_data/dataset_validation')

with open('/content/drive/MyDrive/main_data/en_int2str.json', 'r') as f:
    en_int2str = json.load(f)
with open('/content/drive/MyDrive/main_data/en_str2int.json', 'r') as f:
    en_str2int = json.load(f)
with open('/content/drive/MyDrive/main_data/fr_int2str.json', 'r') as f:
    fr_int2str = json.load(f)
with open('/content/drive/MyDrive/main_data/fr_str2int.json', 'r') as f:
    fr_str2int = json.load(f)

In [2]:
import torch
import torch.nn as nn


# data batching
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example['en_ids'] for example in batch]
        batch_fr_ids = [example['fr_ids'] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_fr_ids = nn.utils.rnn.pad_sequence(batch_fr_ids, padding_value=pad_index)
        batch = {
            'en_ids': batch_en_ids,
            'fr_ids': batch_fr_ids
        }
        return batch
    return collate_fn


def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle
    )
    return data_loader

sos_token = '<sos>'
eos_token = '<eos>'
pad_token = '<pad>'

batch_size = 32 #128
PAD_INDEX = en_str2int[pad_token]

train_data_loader = get_data_loader(dataset_train, batch_size, PAD_INDEX, shuffle=False)
test_data_loader = get_data_loader(dataset_test, batch_size, PAD_INDEX, shuffle=False)
validation_data_loader = get_data_loader(dataset_validation, batch_size, PAD_INDEX, shuffle=False)

result = next(iter(train_data_loader))
result['en_ids'].shape, result['fr_ids'].shape

(torch.Size([7, 32]), torch.Size([24, 32]))

In [3]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_, hidden, cell):
        input_ = input_.unsqueeze(0)
        embedded = self.embedding(input_)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, trg_len):
        '''
        src: [src_len, batch_size]
        trg: [trg_len, batch_size]
        trg_len: length o
        '''
        batch_size = src.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        input_ = trg[0, :]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_, hidden, cell)
            outputs[t] = output
            top1 = output.argmax(1)
            input_ = top1

        return outputs

In [4]:
'''
model hyperparameters
'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = len(en_str2int)
output_dim = len(fr_str2int)

encoder_embedding_dim = 50 #1_000
decoder_embedding_dim = 50 #1_000

hidden_dim = 50 # 1_000
num_layers = 1 # 4


encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    num_layers,
).to(device)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    num_layers,
).to(device)


'''
The paper suggest that initialzing the weight with a uniform distribution of -0.08 to 0.08
'''
for param in encoder.parameters():
    torch.nn.init.uniform_(param, a=-0.08, b=0.08)

for param in decoder.parameters():
    torch.nn.init.uniform_(param, a=-0.08, b=0.08)

model = Seq2Seq(encoder, decoder, device).to(device)
# model = nn.DataParallel(model, device_ids=[1, 2])
criterion = nn.CrossEntropyLoss(ignore_index=fr_str2int['<pad>'])

In [5]:
def train(model, dataloader, optimizer, criterion):
    model.train()

    epoch_loss = 0

    for batch in dataloader:
        src = batch['en_ids'].to(device)
        trg = batch['fr_ids'].to(device)

        optimizer.zero_grad()
        output = model(src, trg, trg.shape[0]).to(device)

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward()

        batch_size = src.shape[1]
        for param in model.parameters():
            param.grad /= batch_size

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [6]:
def evaluate(model, dataloader, criterion):
    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            src = batch['en_ids'].to(device)
            trg = batch['fr_ids'].to(device)

            output = model(src, trg, trg.shape[0])

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [7]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    mins = int(elapsed_time / 60)
    secs = int(elapsed_time - (mins * 60))
    return mins, secs

N_EPOCHS = 7.5
initial_lr = 0.7
optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum=0)


def adjust_learning_rate(optimizer, epoch):
    if epoch >= 5:
        factor = 2 ** ((epoch - 5) * 2)  # Halving every half epoch
        for param_group in optimizer.param_groups:
            param_group['lr'] = initial_lr / factor  # Decay learning rate

total_half_epochs = int(N_EPOCHS * 2)


for i in range(total_half_epochs):
  current_epoch = i / 2

  start_time = time.time()

  if current_epoch >= 5:
      adjust_learning_rate(optimizer, current_epoch)
  current_lr = optimizer.param_groups[0]['lr']

                            # train_data_loader
  train_loss = train(model, test_data_loader, optimizer, criterion) # currently replaced with test_data_loader.
  valid_loss = evaluate(model, validation_data_loader, criterion)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  print(f"Epoch {current_epoch:.1f}: | Learning Rate = {current_lr:.5f} | Time: {epoch_mins}m {epoch_secs}s")
  print(f"\tTrain Loss: {train_loss:.3f}")
  print(f"\tVal. Loss: {valid_loss:.3f}")

Epoch 0.0: | Learning Rate = 0.70000 | Time: 2m 51s
	Train Loss: 11.913
	Val. Loss: 11.899
Epoch 0.5: | Learning Rate = 0.70000 | Time: 2m 42s
	Train Loss: 11.887
	Val. Loss: 11.872
Epoch 1.0: | Learning Rate = 0.70000 | Time: 2m 41s
	Train Loss: 11.861
	Val. Loss: 11.844
Epoch 1.5: | Learning Rate = 0.70000 | Time: 2m 42s
	Train Loss: 11.833
	Val. Loss: 11.815
Epoch 2.0: | Learning Rate = 0.70000 | Time: 2m 42s
	Train Loss: 11.804
	Val. Loss: 11.785
Epoch 2.5: | Learning Rate = 0.70000 | Time: 2m 42s
	Train Loss: 11.774
	Val. Loss: 11.753
Epoch 3.0: | Learning Rate = 0.70000 | Time: 2m 41s
	Train Loss: 11.740
	Val. Loss: 11.716
Epoch 3.5: | Learning Rate = 0.70000 | Time: 2m 41s
	Train Loss: 11.703
	Val. Loss: 11.675
Epoch 4.0: | Learning Rate = 0.70000 | Time: 2m 41s
	Train Loss: 11.659
	Val. Loss: 11.626
Epoch 4.5: | Learning Rate = 0.70000 | Time: 2m 41s
	Train Loss: 11.605
	Val. Loss: 11.565
Epoch 5.0: | Learning Rate = 0.70000 | Time: 2m 41s
	Train Loss: 11.536
	Val. Loss: 11.484

In [16]:
def beam_search_decoder(model, src, beam_width=5, max_len=None):
    model.eval()
    with torch.no_grad():
        hidden, cell = model.encoder(src)

        hidden = hidden.squeeze(1)
        cell = cell.squeeze(1)

        # Use French SOS token since decoder works with French tokens.
        start_token = torch.tensor([fr_str2int['<sos>']], device=device)
        sequences = [(start_token, 0, hidden, cell)]

        # Dynamically set maximum output length based on the source sequence length.
        if max_len is None:
            max_len = src.size(0) * 2  # or any multiplier suitable for your task

        for _ in range(max_len):
            all_candidates = []
            for seq, score, h, c in sequences:
                # Feed the last token of the current sequence.
                output, h_new, c_new = model.decoder(seq[-1], h, c)
                top_k_probs, top_k_words = torch.topk(output, beam_width)

                for i in range(beam_width):
                    # Extend the sequence with the i-th candidate.
                    new_seq = torch.cat((seq, top_k_words[i].unsqueeze(0)))
                    new_score = score + torch.log(top_k_probs[i])
                    all_candidates.append((new_seq, new_score, h_new, c_new))

            # Keep only the best beam_width sequences.
            sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]

            # If the best sequence ends with EOS, break early.
            if sequences[0][0][-1].item() == fr_str2int['<eos>']:
                break

        return sequences[0][0]

In [19]:
def translate_sentence(model, sentence, en_str2int, fr_int2str, beam_width=5, max_len=None):
    """
    Translate a source sentence using beam search.
    - sentence: a string (or list) of tokens in the source language.
    - en_str2int: dictionary mapping source tokens to indices.
    - fr_int2str: dictionary mapping target indices to tokens.
    - beam_width: beam width to use during decoding.
    - max_len: maximum output length; if None, it is set dynamically.
    """
    model.eval()

    # Tokenize and convert sentence to indices.
    tokens = sentence.strip().split()  # Simple whitespace split; adjust as needed.
    # Prepend SOS if needed (depends on how you encoded your training data).
    src_indices = [en_str2int['<sos>']] + [en_str2int.get(token, en_str2int['<pad>']) for token in tokens] + [en_str2int['<eos>']]
    src_tensor = torch.LongTensor(src_indices).unsqueeze(1).to(device)  # Shape: [src_len, 1]
    print(src_tensor.shape)
    # Run beam search decoder.
    output_indices = beam_search_decoder(model, src_tensor, beam_width, max_len)

    # Convert output indices to tokens.
    # Optionally, remove the SOS and EOS tokens from the output.
    # Handle unknown tokens by replacing them with <unk>.
    output_tokens = [fr_int2str.get(idx.item(), '<unk>') for idx in output_indices]
    return output_tokens

In [27]:
sample_sentence = "this is a test"
translated_tokens = translate_sentence(model, sample_sentence, en_str2int, fr_int2str, beam_width=12)
print("Translated:", " ".join(translated_tokens))

torch.Size([6, 1])
Translated: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


### Evaluation

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

def calculate_bleu(data, model, device, max_len=50):
    trgs = []
    pred_trgs = []

    for datum in data:
        src = datum['en_ids']
        trg = datum['fr_ids']

        src_sentence = src.tolist()[0]
        trg_sentence = trg.tolist()[0]

        pred_tokens = translate_sentence(src_sentence, model.encoder, model.decoder, en_str2int, fr_str2int, device, max_len)
        pred_tokens = pred_tokens[1:-1]  # Remove <sos> and <eos>

        trg_tokens = [int2fr[token] for token in trg_sentence if token not in [fr_str2int['<sos>'], fr_str2int['<eos>'], fr_str2int['<pad>']]]

        trgs.append([trg_tokens])
        pred_trgs.append(pred_tokens)

    return corpus_bleu(trgs, pred_trgs)

calculate_bleu(validation_data_loader, model, device=device)

### Save and load model

In [ ]:
"""torch.save({
    'encoder_state_dict': encoder.state_dict(),
    'decoder_state_dict': decoder.state_dict(),
}, 'seq2seq_model.pth')

checkpoint = torch.load('seq2seq_model.pth')
encoder.load_state_dict(checkpoint['encoder_state_dict'])
decoder.load_state_dict(checkpoint['decoder_state_dict'])
model = Seq2Seq(encoder, decoder, device).to(device)
model.eval()"""

In [26]:
torch.save(model.state_dict(), '/content/drive/MyDrive/main_data/model_weights.pth')

In [25]:
model__ = Seq2Seq(encoder, decoder, device).to(device)


model__.load_state_dict(torch.load('/content/drive/MyDrive/main_data/model_weights.pth', weights_only=True))
model__.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(126272, 50)
    (rnn): LSTM(50, 50)
  )
  (decoder): Decoder(
    (embedding): Embedding(150345, 50)
    (rnn): LSTM(50, 50)
    (fc_out): Linear(in_features=50, out_features=150345, bias=True)
  )
)

In [ ]:
sample_sentence = "this is a test"
translated_tokens = translate_sentence(model__, sample_sentence, en_str2int, fr_int2str, beam_width=12)
print("Translated:", " ".join(translated_tokens))